In [ ]:
from bs4 import BeautifulSoup
import re
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
def search():
    input_search = input("inter search : ")
    location = input("inter your location 'defult is Egypt': ")
    
    search = input_search.split()
    if not location:
        location = "Egypt"
    
    if len(search) == 2:
        link = f"https://eg.indeed.com/jobs?q={search[0]}+{search[-1]}&l=&vjk=05b4c288eeceaf5c"


    elif len(search) == 3:
        link = f"https://eg.indeed.com/jobs?q={search[0]}+{search[1]}+{search[-1]}&l=&vjk=9db6ce606226a6e3"


    elif len(search) == 4:
        link = f"https://eg.indeed.com/jobs?q={searsh[0]}+{searsh[1]}+{searsh[2]}+{searsh[-1]}&l=&vjk=74f253bd36042d58"

    else:
        link = f"https://eg.indeed.com/jobs?q={''.join(search)}&l=&vjk=11de4a49c29f636c"


    print(location)
    return link, input_search, location

In [ ]:
# open chromedriver on your PC
def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions # Pictures are not displayed in page
                                                         # To download components quickly
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
    from webdriver_manager.chrome import ChromeDriverManager
        
    global web
    chromedriver_installer = Service(ChromeDriverManager().install())
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2} # Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
        
    return web


In [ ]:
link, input_search, location = search()
web = open_chromedriver()

In [ ]:
# scroll down all the page to load all data of item                       
def scroll_down_all_the_way():
    
    from time import sleep 
    
    end = 0
    while end < web.execute_script('return document.body.scrollHeight'):
        start = end
        end = start + 700
        web.execute_script(f'window.scrollTo({start}, {end})')
        sleep(4)
        
        try:
            specs = web.find_element(by="xpath", value="/html/body/div[1]/div/main/section/button").click()
        except Exception as error:
            pass

In [ ]:
def get_links():
    web.get(link) # open link
    soup = BeautifulSoup(web.page_source, 'html.parser') # send request

    links = []
    
    while True:
        try:
            specs = web.find_element(by="xpath", value="/html/body/main/div/div[1]/div/div[5]/div[1]/nav/ul/li[6]/a").click()
            jobs_data = soup.find("ul", {"class": "css-zu9cdh eu4oa1w0"})
            job_item = jobs_data.find_all("li", {"class": "css-5lfssm eu4oa1w0"})

            for i in job_item:
                try:
                    links.append("https://eg.indeed.com" + i.a["href"])
                except:
                    pass
            time.sleep(2)
        except:
            try:
                web.find_element(by="xpath", value="/html/body/div[2]/div[1]/div[1]/button").click()
            except:
                break
    return links

In [ ]:
links = get_links()
len(links)

In [ ]:
def display(web = web):
    time.sleep(25)
    WebDriverWait(web, 60).until(ec.visibility_of_element_located((By.XPATH, '/html/body'))) # element for search results
    time.sleep(10)

In [ ]:
def get_information(links=links):
    listings = []
    for link in links:
        try:
            web.get(link)
            time.sleep(165)
    #         display()
            soup = BeautifulSoup(web.page_source, 'html.parser') # send request

            listing = {}

            contant = soup.find("div", {"class": "jobsearch-JobComponent css-u4y1in eu4oa1w0 jobsearch-JobComponent-bottomDivider"})   
            listing["job_title"] = contant.find("div", {"class": "jobsearch-JobInfoHeader-title-container jobsearch-JobInfoHeader-title-containerEji css-82ysm6 eu4oa1w0"}).get_text(strip=True)
            listing["company_name"] = contant.find("span", {"class": "css-1cxc9zk e1wnkr790"}).get_text(strip=True)
            listing["city_name"] = contant.find("div", {"data-testid": "inlineHeader-companyLocation"}).get_text(strip=True)

            job_details = contant.find("div", {"class": "css-1bkjm6j e37uo190"})
            if job_details:
                listing["job_details"] = job_details.get_text(strip=True)

            job_type = contant.find("div", {"class": "css-1hplm3f eu4oa1w0"})
            if job_type:
                listing["job_type"] = job_type.get_text(strip=True)
            else:
                listing["job_type"] = "Full time"

            job_info = contant.find("div", {"id": "jobDescriptionText"})
            job_decrip_and_req = [i.get_text(strip=True) for i in job_info.find_all("ul")]
            listing["job_description"] = job_decrip_and_req[0]
            listing["job_requirements"] = job_decrip_and_req[1]
            if len(job_decrip_and_req) > 2:
                listing["additional_info"] = job_decrip_and_req[2]
            listing["company_info"] = job_info.get_text(strip=True)[:-len("".join(job_decrip_and_req))]

            listings.append(listing)
            print("\n", "-"*100, listing, "\n")
            
        except Exception as error:
            print(error)
        
    return listings

In [ ]:
listings = get_information()

In [ ]:
len(listings)

In [ ]:
import pandas as pd
pd.options.display.max_columns = None # donot display all rows and columns of data

df = pd.DataFrame.from_records(listings)
df

In [ ]:
df.isna().sum()

In [ ]:
def save(path, df=df, save_as_csv=True, data=listings):
    import pandas as pd
    import joblib


    joblib.dump(listings, path) # load data form PC
    
    if save_as_csv:
        df.to_csv(path, index=False)

In [ ]:
file_name = "Indeed_" + input_search
path = f"F:\\NLP Apple Course\\NLP Project\\{file_name}.pkl"

save(path = path)

In [ ]:
df = pd.DataFrame.from_records(listings)
df.to_csv(path, index=False)

In [ ]:
csv_path = f"F:\\NLP Apple Course\\NLP Project\\{file_name}.csv"

df.to_csv(csv_path, index=False)

In [ ]:
listings

In [ ]:
path